# Analysis of crane and load velocities and accelerations

In [1]:
#set up code for notebook, importing relevant modules

%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from scipy import integrate
from ipywidgets import interact, fixed

In [2]:
distance = 10.0 # [m]
g = 9.81 # [m/s^2]
length = 3.0 # [m]
mass = 500.0 #[kg]

upperOmega = 6.0
lowerOmega = 1.0
omega = 2.1

In [3]:
def plotInputFunc(omega=2.1):
    """plot the input function and vary omega"""
    t = np.linspace(0.0, np.pi/lowerOmega, 100)
    x = distance/2.0*(1.0-np.cos(omega*t))
    fig, axes = plt.subplots(figsize=(16,5))
    axes.plot(t,x)
    plt.axvline(x=np.pi/omega)
    plt.show()
    return

#interact(plotInputFunc, omega=(lowerOmega,upperOmega,0.1));

In [6]:
lowerOmega = 1.5
upperOmega = 3.6
lowerLength = 0.5
upperLength = 10
lowerDistance = 0.5
upperDistance = 10
lowerMass = 0
upperMass = 1000

def plotRotationalChar(omega=2.1, length=3, distance=10, mass=500):
    #find the rotational characteristics of motion phi, phiDot and phiDotDot before point B  
    #initialise parameters

    timeBeforeB = np.linspace(0.0,np.pi/lowerOmega,100)
    t0, t1 = 0, np.pi/lowerOmega
    t = timeBeforeB
    phiInit = [0,0]
    phi = np.zeros((len(t), len(phiInit)))
    phi[0,:] = phiInit
    xDotDot = omega**2 * distance/2 *np.cos(omega*t)
    
    def odeBeforeB(t,phi):
    #calculate phi values for use in the ODE45 equivalent before the crane reaches B
        xDotDot = omega**2 * distance/2 * np.cos(omega*t)
        return [phi[1], (-g * phi[0] - xDotDot)/length]
    
    r = integrate.ode(odeBeforeB).set_integrator("dopri5")
    r.set_initial_value(phiInit, t0)
    
    for i in range(1, t.size):
        phi[i, :] = r.integrate(t[i])
        if not r.successful():
           raise RuntimeError("Could not integrate")

    phiDotDot = (-g * phi[:,0] - xDotDot)/length
    
    #plot the rotational characteristics of motion phi, phiDot and phiDotDot before point B
    fig, axes = plt.subplots(figsize=(12,8))
    axes.plot(t, phi[:,0], 'c', t, phi[:,1], 'm', t, phiDotDot, 'b')
    plt.axvline(x=np.pi/omega)
    plt.axvline(x=0)
    plt.axhline(y=50)
    plt.axhline(y=-50)
    plt.legend(['phi','phiDot','phiDotDot'])
    plt.show()
    return

def plotChar():
    interact(plotRotationalChar, omega=(lowerOmega,upperOmega,0.1), length=(lowerLength, upperLength,0.1), distance=(lowerDistance,upperDistance,0.1), mass=(lowerMass,upperMass,0.1), continuous_update=False);
    return 

#  Plotting some stuff

In [7]:
plotChar()

interactive(children=(FloatSlider(value=2.1, description='omega', max=3.6, min=1.5), FloatSlider(value=3.0, description='length', max=10.0, min=0.5), FloatSlider(value=10.0, description='distance', max=10.0, min=0.5), FloatSlider(value=500.0, description='mass', max=1000.0), Output()), _dom_classes=('widget-interact',))